**This notebook is used to upload saved data from multiple .csv files to the influx database on a given webserver**

In [ ]:
import struct
import requests

#Returns a string representing the hex of an 8 bit integer
def print8Bit(num):
    return "%0.2X" % num

#Returns a string representing the hex of a 16 bit integer
def print16Bit(num):
    return "%0.2X%0.2X" % ((num & 0xFF), (num >> 8))

#Returns a string representing the hex of a 32 bit integer
def print32Bit(num):
    return "%0.2X%0.2X%0.2X%0.2X" % (num & 0xFF, (num >> 8) & 0xFF, (num >> 16) & 0xFF, (num >> 24) & 0xFF)

#Returns a string representing the hex of a 32 bit float
def printFloat(num):
    #Taken from https://stackoverflow.com/questions/23624212/how-to-convert-a-float-into-hex
    hexStr = hex(struct.unpack('<I', struct.pack('<f', num))[0])
    
    #Add 0s if necessary
    while len(hexStr) < 10:
        hexStr += '0'
        
    return hexStr[8:10] + hexStr[6:8] + hexStr[4:6] + hexStr[2:4]

#Count the number of data points that have a location
def countLocations(points):
    ans = 0
    for i in points:
        if (i.split(','))[1] != '':
            ans += 1
    
    return ans

#Get the data to post to the server
def getData(filePath):
    #Read the data to send from ToSend.csv
    toSend = open(filePath + "ToSend.csv", "r")
    toSendLines = toSend.readlines()
    toSend.close()
    
    #How many nodes there are
    ans = print8Bit(len(toSendLines) - 1)
    
    #Loop through every node that has data to send
    for i in toSendLines[1:]:
        #Split the current line from ToSend.csv at the commas
        curr = (i.strip()).split(',')
        
        #Add the node's ID
        ans += print16Bit(int(curr[0]))
        
        #Open the node's file
        nodeFile = open(filePath + "node" + str(int(curr[0])) + ".csv")
        
        #The first two lines are special
        firstLine = (nodeFile.readline()).strip()
        secondLine = (nodeFile.readline()).strip()
        
        #Read the data that needs to be sent
        nodeFile.seek(int(curr[2]))
        data = nodeFile.readlines()
        
        nodeFile.close()
        
        #Add the number of locations
        locations = countLocations(data)
        ans += print8Bit(locations)
        
        if (locations != 0):
            #Add the node's name
            ans += print8Bit(len(firstLine))
            for c in firstLine:
                ans += print8Bit(ord(c))
        
            #Add the number (index) of the data points that have a location
            for j in range(int(curr[1])):
                if (data[j].split(','))[1] != '':
                    ans += print8Bit(j + 1)
        
        #Add the number of sensors
        sensors = secondLine.split(',')
        ans += print8Bit(len(sensors) - 3)
        
        #Add the sensor names
        for sensor in sensors[3:]:
            ans += print8Bit(len(sensor))
            for c in sensor:
                ans += print8Bit(ord(c))
        
        #Add the number of data points
        ans += print8Bit(len(data))
        
        #Loop through every data point
        for point in data:
            #Split the current data point at the commas
            curr = (point.strip()).split(',')
            
            #Add the time
            ans += print32Bit(int(curr[0]) & 0xFFFFFFFF)
            
            #If it has a location, add it
            if curr[1] != '':
                ans += printFloat(float(curr[1]))
                ans += printFloat(float(curr[2]))
            
            #Add the sensor values
            for value in curr[3:]:
                ans += printFloat(float(value))
    
    
    return ans

#-------------------------------------------------

#Get URL
URL = input("Website URL: ")
if (URL[-1] != '/'):
    URL += '/'
URL += 'sensor/data'

#Get file path
filePath = input("Relative path to .csv files: ")
if (filePath != '' && filePath[-1] != '/'):
    filePath += '/'

#Get the data from the .csv files
params = {'data':getData(filePath)}

#Post the data to the server
requests.post(url = URL, params = params)

#URL for current setup: https://www.cas.mcmaster.ca/ollie